In [1]:
!git clone https://github.com/Tanveer2719/NIDS_Coursework.git
%cd /kaggle/working/NIDS_Coursework/My_Code
!ls /kaggle/input


Cloning into 'NIDS_Coursework'...
remote: Enumerating objects: 262, done.
remote: Counting objects: 100% (262/262), done.
remote: Compressing objects: 100% (171/171), done.
remote: Total 262 (delta 126), reused 221 (delta 85), pack-reused 0 (from 0)
Receiving objects: 100% (262/262), 1.13 MiB | 21.81 MiB/s, done.
Resolving deltas: 100% (126/126), done.
/kaggle/working/NIDS_Coursework/My_Code
UNSW-NB15_combined.csv


In [2]:
import os
import random
import numpy as np
import tensorflow as tf

def set_seed(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.keras.utils.set_random_seed(seed)
    tf.config.experimental.enable_op_determinism()

set_seed(42)

2025-08-27 12:54:44.414676: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756299284.616094      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756299284.673231      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
import os
os.listdir("/kaggle/input")


['UNSW-NB15_combined.csv']

In [7]:
import pandas as pd
import numpy as np

path = "/kaggle/input/UNSW-NB15_combined.csv"

df = pd.read_csv(path)

print(df.shape)
df = df.drop(columns=['id', 'label'])
print(df.shape)

print(f"🧾 Unique class labels: {df['attack_cat'].unique()}")
df['label'] = df['attack_cat'].apply(lambda x: 0 if x == 'Normal' else 1)

print(f'Unique values for label {df["label"].unique()}')

df = df.drop(columns=["attack_cat"])
print(f'new shape{df.shape}')



# --------------------------- perform preprocessing--------------

from preprocess import Preprocess

# separate the features and labels so that the labesl are not encoded
labels = df['label']
features = df.drop(columns=['label'])

pp = Preprocess()
processed_features = pp.fit_transform_df_auto(df = features,n_categorical_levels=32, expected_categorical_format='onehot')

print(processed_features.shape)

# concat the features and the labels
processed_full_df = pd.concat([processed_features, labels], axis=1)

# Filter out categorical one-hot encoded columns
numerical_columns = [col for col in processed_full_df 
                     if not (col.startswith("proto_") or 
                             col.startswith("service_") or 
                             col.startswith("state_") or
                             col == 'label')]

target = processed_full_df['label']
features = processed_full_df.drop(columns=['label'])

(257673, 45)
(257673, 43)
🧾 Unique class labels: ['Normal' 'Backdoor' 'Analysis' 'Fuzzers' 'Shellcode' 'Reconnaissance'
 'Exploits' 'DoS' 'Worms' 'Generic']
Unique values for label [0 1]
new shape(257673, 43)
Encoding the 32 levels for proto
Encoding the 13 levels for service
Encoding the 11 levels for state
(257673, 95)


In [8]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

X = features.astype("float32").values  # (257673, 95) as float32
y = target.astype("int32").values      # (257673,)


# reshape for (samples, timesteps, features=1)
X = np.expand_dims(X, axis=-1)  # (257673, 95, 1)

# Split train, val, test
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.2, random_state=42, stratify=y_temp)

print(f"X_train: {X_train.shape}")
print(f"y_train: {y_train.shape}")
print(f"X_val: {X_val.shape}")
print(f"y_val: {y_val.shape}")
print(f"X_test: {X_test.shape}")
print(f"y_test: {y_test.shape}")



input_shape = (X_train.shape[1], X_train.shape[2])


X_train: (206138, 95, 1)
y_train: (206138,)
X_val: (41228, 95, 1)
y_val: (41228,)
X_test: (10307, 95, 1)
y_test: (10307,)


In [9]:
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

def evaluate_metrics(y_true, y_pred):
    f1 = f1_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)

    if cm.shape == (2, 2):
        tn, fp, fn, tp = cm.ravel()
        numerator = tp * tn - fp * fn
        denominator = ((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)) ** 0.5
        mcc = numerator / (denominator + 1e-10)
        specificity = tn / (tn + fp + 1e-10)
        balanced_acc = (recall + specificity) / 2
    else:
        mcc = 0
        balanced_acc = 0
        specificity = 0

    return {
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'mcc': mcc,
        'specificity': specificity,
        'balanced_accuracy': balanced_acc
    }

## Model Building

In [10]:
from tensorflow.keras import layers, models, optimizers, losses, metrics

# LSTM baseline model
model = models.Sequential([
    layers.LSTM(64, return_sequences=False, input_shape=input_shape),  # LSTM instead of GRU
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=256,
    validation_data=(X_val, y_val),
    verbose=1
)

# Predictions
y_pred = (model.predict(X_test) > 0.5).astype("int32")

# Evaluate with your custom function
normal_results = evaluate_metrics(y_true=y_test, y_pred=y_pred)
print(f'Baseline LSTM results: {normal_results}')


I0000 00:00:1756299370.427852      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1756299370.428584      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20


2025-08-27 12:56:12.290467: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_15}}
I0000 00:00:1756299374.760361     114 cuda_dnn.cc:529] Loaded cuDNN version 90300


799/806 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6524 - loss: 0.6322

2025-08-27 12:56:21.353923: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_15}}


806/806 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - accuracy: 0.6529 - loss: 0.6314 - val_accuracy: 0.8452 - val_loss: 0.3885
Epoch 2/20
806/806 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.8667 - loss: 0.3190 - val_accuracy: 0.8876 - val_loss: 0.2582
Epoch 3/20
806/806 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.8844 - loss: 0.2639 - val_accuracy: 0.8860 - val_loss: 0.2611
Epoch 4/20
806/806 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.8895 - loss: 0.2495 - val_accuracy: 0.8870 - val_loss: 0.2543
Epoch 5/20
806/806 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.8910 - loss: 0.2451 - val_accuracy: 0.8931 - val_loss: 0.2295
Epoch 6/20
806/806 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.8950 - loss: 0.2345 - val_accuracy: 0.8993 - val_loss: 0.2224
Epoch 7/20
806/806 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.8966 - loss: 0.2271 - val_accuracy: 0.8912 - val_loss: 0.2353
Epoch 8/20
806/806 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.8966 - loss: 0.2257 - val_accuracy: 0.8802 - val

2025-08-27 12:58:17.825075: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Baseline LSTM results: {'f1': 0.9315194835139496, 'precision': 0.9433374844333748, 'recall': 0.9199939274328223, 'mcc': 0.8148487010682461, 'specificity': 0.9021505376343844, 'balanced_accuracy': 0.9110722325336034}


## Attack codes

In [11]:
import tensorflow as tf

@tf.function
def fgsm_attack_tf(model, inputs, labels, epsilon=0.05, clip_min=-1.5, clip_max=1.5):
    inputs = tf.convert_to_tensor(inputs, dtype=tf.float32)
    labels = tf.reshape(tf.convert_to_tensor(labels, dtype=tf.float32), (-1, 1))

    with tf.GradientTape() as tape:
        tape.watch(inputs)
        predictions = model(inputs, training=False)
        loss = tf.keras.losses.binary_crossentropy(labels, predictions)
        loss = tf.reduce_mean(loss)

    gradients = tape.gradient(loss, inputs)
    signed_grad = tf.sign(gradients)
    x_adv = inputs + epsilon * signed_grad
    x_adv = tf.clip_by_value(x_adv, clip_min, clip_max)
    return x_adv


@tf.function
def pgd_attack_tf(model, inputs, labels, epsilon=0.05, alpha=0.01, num_iter=10, clip_min=-1.5, clip_max=1.5):
    inputs = tf.convert_to_tensor(inputs, dtype=tf.float32)
    labels = tf.reshape(tf.convert_to_tensor(labels, dtype=tf.float32), (-1, 1))

    x_adv = tf.identity(inputs)

    for _ in tf.range(num_iter):
        with tf.GradientTape() as tape:
            tape.watch(x_adv)
            predictions = model(x_adv, training=False)
            loss = tf.keras.losses.binary_crossentropy(labels, predictions)
            loss = tf.reduce_mean(loss)

        gradients = tape.gradient(loss, x_adv)
        x_adv = x_adv + alpha * tf.sign(gradients)
        x_adv = tf.clip_by_value(x_adv, inputs - epsilon, inputs + epsilon)
        x_adv = tf.clip_by_value(x_adv, clip_min, clip_max)

    return x_adv


@tf.function
def mi_fgsm_attack(model, inputs, labels, epsilon=0.05, alpha=0.01, steps=40, decay=1.0, clip_min=-1.5, clip_max=1.5):
    inputs = tf.convert_to_tensor(inputs, dtype=tf.float32)
    labels = tf.reshape(tf.convert_to_tensor(labels, dtype=tf.float32), (-1, 1))

    x_adv = tf.identity(inputs)
    g = tf.zeros_like(inputs)

    for _ in tf.range(steps):
        with tf.GradientTape() as tape:
            tape.watch(x_adv)
            logits = model(x_adv, training=False)
            loss = tf.keras.losses.binary_crossentropy(labels, logits)
            loss = tf.reduce_mean(loss)

        grad = tape.gradient(loss, x_adv)
        # Normalize gradient
        grad_norm = tf.reduce_mean(tf.abs(grad), axis=list(range(1, len(grad.shape))), keepdims=True)
        grad = grad / (grad_norm + 1e-8)

        g = decay * g + grad
        x_adv = x_adv + alpha * tf.sign(g)
        x_adv = tf.clip_by_value(x_adv, inputs - epsilon, inputs + epsilon)
        x_adv = tf.clip_by_value(x_adv, clip_min, clip_max)

    return x_adv


In [12]:
def evaluate_model(model, X, y,epsilon_list=[0.01, 0.03, 0.05, 0.07]):
    import tensorflow as tf
    import numpy as np

    X_clean = X
    x_clean_tf = tf.convert_to_tensor(X_clean, dtype=tf.float32)
    y_tf = tf.convert_to_tensor(y, dtype=tf.float32)

    # ---------------- Clean Evaluation ----------------
    print("✅ Clean Evaluation:")
    y_pred_clean = (model.predict(x_clean_tf) > 0.5).astype(int).flatten()
    clean_metrics = evaluate_metrics(y, y_pred_clean)
    print(f"  F1: {clean_metrics['f1']:.4f}, Precision: {clean_metrics['precision']:.4f}, Recall: {clean_metrics['recall']:.4f}, MCC: {clean_metrics.get('mcc', 'N/A')}, Specificity: {clean_metrics.get('specificity', 'N/A')}, Balanced Acc: {clean_metrics.get('balanced_accuracy', 'N/A')}")

    # ---------------- FGSM Evaluation ----------------
    print("\nFGSM Attack Evaluation")
    fgsm_results = []
    for eps in epsilon_list:
        x_adv = fgsm_attack_tf(model, x_clean_tf, y_tf, epsilon=eps )
        x_adv_tf = tf.convert_to_tensor(x_adv.numpy(), dtype=tf.float32)
        y_pred_adv = (model.predict(x_adv_tf) > 0.5).astype(int).flatten()

        adv_metrics = evaluate_metrics(y, y_pred_adv)
        fgsm_results.append({'epsilon': eps, 'adversarial': adv_metrics})

        print(f"  FGSM ε={eps:.2f} → F1: {adv_metrics['f1']:.4f}, Precision: {adv_metrics['precision']:.4f}, Recall: {adv_metrics['recall']:.4f}, MCC: {adv_metrics.get('mcc', 'N/A')}, Specificity: {adv_metrics.get('specificity', 'N/A')}, Balanced Acc: {adv_metrics.get('balanced_accuracy', 'N/A')}")

    # ---------------- PGD Evaluation ----------------
    print("\nPGD Attack Evaluation:")
    pgd_results = []
    for eps in epsilon_list:
        alpha = eps * 0.2
        x_adv = pgd_attack_tf(model, x_clean_tf, y_tf, epsilon=eps, alpha=alpha, num_iter=10)
        x_adv_tf = tf.convert_to_tensor(x_adv.numpy(), dtype=tf.float32)
        y_pred_adv = (model.predict(x_adv_tf) > 0.5).astype(int).flatten()

        adv_metrics = evaluate_metrics(y, y_pred_adv)
        pgd_results.append({'epsilon': eps, 'adversarial': adv_metrics})

        print(f"  PGD ε={eps:.2f} → F1: {adv_metrics['f1']:.4f}, Precision: {adv_metrics['precision']:.4f}, Recall: {adv_metrics['recall']:.4f}, MCC: {adv_metrics.get('mcc', 'N/A')}, Specificity: {adv_metrics.get('specificity', 'N/A')}, Balanced Acc: {adv_metrics.get('balanced_accuracy', 'N/A')}")

    # ---------------- MI-FGSM Evaluation ----------------
    print("\nMI-FGSM Attack Evaluation:")
    mifgsm_results = []
    for eps in epsilon_list:
        alpha = eps * 0.2
        steps = 10   
        decay = 1.0   

        x_adv = mi_fgsm_attack(model, x_clean_tf, y_tf, epsilon=eps, alpha=alpha, steps=steps, decay=decay)
        x_adv_tf = tf.convert_to_tensor(x_adv.numpy(), dtype=tf.float32)
        y_pred_adv = (model.predict(x_adv_tf) > 0.5).astype(int).flatten()

        adv_metrics = evaluate_metrics(y, y_pred_adv)
        mifgsm_results.append({'epsilon': eps, 'adversarial': adv_metrics})

        print(f" MI-FGSM ε={eps:.2f} → F1: {adv_metrics['f1']:.4f}, Precision: {adv_metrics['precision']:.4f}, Recall: {adv_metrics['recall']:.4f}, MCC: {adv_metrics.get('mcc', 'N/A')}, Specificity: {adv_metrics.get('specificity', 'N/A')}, Balanced Acc: {adv_metrics.get('balanced_accuracy', 'N/A')}")

    return fgsm_results, pgd_results, mifgsm_results
        


In [13]:
print("Results: ")
epsilons = [.01,.03,.05,.07]
fgsm_results, pgd_results, mifgsm_results = evaluate_model(
    model=model,
    X=X_test,
    y=y_test,
    epsilon_list=epsilons,
)



CNN Results: 
✅ Clean Evaluation:
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
  F1: 0.9315, Precision: 0.9433, Recall: 0.9200, MCC: 0.8148487010682461, Specificity: 0.9021505376343844, Balanced Acc: 0.9110722325336034

FGSM Attack Evaluation
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
  FGSM ε=0.01 → F1: 0.8096, Precision: 0.8292, Recall: 0.7910, MCC: 0.49470735975771524, Specificity: 0.7115591397849271, Balanced Acc: 0.7512555073450216
 69/323 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

2025-08-27 12:58:23.312968: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
  FGSM ε=0.03 → F1: 0.4563, Precision: 0.6077, Recall: 0.3653, MCC: -0.051551681195194016, Specificity: 0.5825268817204144, Balanced Acc: 0.4738958987317724
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
  FGSM ε=0.05 → F1: 0.3298, Precision: 0.4729, Recall: 0.2532, MCC: -0.24953053014296436, Specificity: 0.5002688172042876, Balanced Acc: 0.37674743425874013
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
  FGSM ε=0.07 → F1: 0.2853, Precision: 0.3705, Recall: 0.2320, MCC: -0.45669682210371954, Specificity: 0.3021505376344005, Balanced Acc: 0.26706130191269134

PGD Attack Evaluation:
 68/323 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

2025-08-27 12:58:30.282262: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
  PGD ε=0.01 → F1: 0.7876, Precision: 0.8066, Recall: 0.7694, MCC: 0.43590526428415777, Specificity: 0.6733870967741754, Balanced Acc: 0.7213906790990963
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
  PGD ε=0.03 → F1: 0.0897, Precision: 0.1822, Recall: 0.0595, MCC: -0.48873134437779553, Specificity: 0.5268817204300934, Balanced Acc: 0.29319643938614126
 68/323 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

...

323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
  PGD ε=0.05 → F1: 0.0710, Precision: 0.1393, Recall: 0.0477, MCC: -0.550537994825748, Specificity: 0.4784946236559011, Balanced Acc: 0.2630821380007151
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
  PGD ε=0.07 → F1: 0.0483, Precision: 0.0871, Recall: 0.0334, MCC: -0.6550658945892001, Specificity: 0.3798387096774091, Balanced Acc: 0.2066189145775842

MI-FGSM Attack Evaluation:
 68/323 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

2025-08-27 12:58:46.151039: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
 MI-FGSM ε=0.01 → F1: 0.7921, Precision: 0.8115, Recall: 0.7736, MCC: 0.4482339813393614, Specificity: 0.6817204301075086, Balanced Acc: 0.7276827442779839
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
 MI-FGSM ε=0.03 → F1: 0.0973, Precision: 0.1952, Recall: 0.0648, MCC: -0.4798426125121016, Specificity: 0.5266129032257922, Balanced Acc: 0.295718778924267
 66/323 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

2025-08-27 12:58:54.135823: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
 MI-FGSM ε=0.05 → F1: 0.0698, Precision: 0.1373, Recall: 0.0468, MCC: -0.5512297566776413, Specificity: 0.4795698924731054, Balanced Acc: 0.26316432987098415
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
 MI-FGSM ε=0.07 → F1: 0.0491, Precision: 0.0800, Recall: 0.0354, MCC: -0.7302671130658097, Specificity: 0.2801075268817129, Balanced Acc: 0.15774011534612442


## Adversarial Training

In [14]:
import tensorflow as tf
from tensorflow.keras import Model

class PGDAdversarialTrainerCNN(tf.keras.Model):
    def __init__(self, base_model, epsilon_range=(0.01, 0.1), alpha=0.01, num_iter=7, clip_min=-1.5, clip_max=1.5):
        super().__init__()
        self.base_model = base_model
        self.epsilon_range = epsilon_range
        self.alpha = alpha
        self.num_iter = num_iter
        self.clip_min = clip_min
        self.clip_max = clip_max
        self.loss_fn = tf.keras.losses.BinaryCrossentropy()
        self.metric = tf.keras.metrics.BinaryAccuracy()

    def compile(self, optimizer, **kwargs):
        super().compile(**kwargs)
        self.optimizer = optimizer

    def pgd_attack_batch(self, x, y):
        x_adv = tf.identity(x)
        y = tf.expand_dims(tf.cast(y, tf.float32), axis=-1)
        epsilon = tf.random.uniform([], *self.epsilon_range)

        for _ in range(self.num_iter):
            with tf.GradientTape() as tape:
                tape.watch(x_adv)
                preds = self.base_model(x_adv, training=True)
                loss = self.loss_fn(y, preds)

            grad = tape.gradient(loss, x_adv)
            x_adv = x_adv + self.alpha * tf.sign(grad)
            # Project back to epsilon ball around original input
            x_adv = tf.clip_by_value(x_adv, x - epsilon, x + epsilon)
            x_adv = tf.clip_by_value(x_adv, self.clip_min, self.clip_max)

        return x_adv

    def train_step(self, data):
        x, y = data
        x_adv = self.pgd_attack_batch(x, y)

        # Combine clean and adversarial samples
        x_combined = tf.concat([x, x_adv], axis=0)
        y_combined = tf.concat([y, y], axis=0)
        y_combined = tf.expand_dims(tf.cast(y_combined, tf.float32), axis=-1)

        with tf.GradientTape() as tape:
            preds = self.base_model(x_combined, training=True)
            loss = self.loss_fn(y_combined, preds)

        grads = tape.gradient(loss, self.base_model.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.base_model.trainable_variables))
        self.metric.update_state(y_combined, preds)

        return {"loss": loss, "accuracy": self.metric.result()}

    def test_step(self, data):
        x, y = data
        y = tf.expand_dims(tf.cast(y, tf.float32), axis=-1)
        preds = self.base_model(x, training=False)
        loss = self.loss_fn(y, preds)
        self.metric.update_state(y, preds)
        return {"loss": loss, "accuracy": self.metric.result()}

    def call(self, inputs):
        return self.base_model(inputs)


In [15]:
# Wrap your existing CNN

from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor='val_loss',   # monitor validation loss
    patience=3,           # stop if no improvement for 3 epochs
    restore_best_weights=True
)


adv_model = PGDAdversarialTrainerCNN(
    base_model=model,           
    epsilon_range=(0.01, 0.05),
    alpha=0.01,
    num_iter=7,
    clip_min=-1.5,
    clip_max=1.5
)

adv_model.compile(optimizer=tf.keras.optimizers.Adam())

# Train with adversarial training
adv_model.fit(
    x=X_train,
    y=y_train,
    validation_data=(X_val, y_val),
    batch_size=256,
    epochs=20,
    callbacks=[early_stop] 

)

print("Advesarial training Results: ")
epsilons = [.01,.03,.05,.07]
fgsm_results, pgd_results, mifgsm_results = evaluate_model(
    model=adv_model,
    X=X_test,
    y=y_test,
    epsilon_list=epsilons,
)


Epoch 1/20


2025-08-27 12:58:59.394901: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_15}}


805/806 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.7412 - loss: 0.4932

2025-08-27 12:59:40.186812: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_15}}


806/806 ━━━━━━━━━━━━━━━━━━━━ 42s 43ms/step - accuracy: 0.7413 - loss: 0.4932 - val_accuracy: 0.8714 - val_loss: 0.2320
Epoch 2/20
806/806 ━━━━━━━━━━━━━━━━━━━━ 34s 42ms/step - accuracy: 0.8501 - loss: 0.3417 - val_accuracy: 0.8867 - val_loss: 0.1800
Epoch 3/20
806/806 ━━━━━━━━━━━━━━━━━━━━ 34s 42ms/step - accuracy: 0.8736 - loss: 0.3083 - val_accuracy: 0.8878 - val_loss: 0.1378
Epoch 4/20
806/806 ━━━━━━━━━━━━━━━━━━━━ 34s 42ms/step - accuracy: 0.8816 - loss: 0.2857 - val_accuracy: 0.8896 - val_loss: 0.1155
Epoch 5/20
806/806 ━━━━━━━━━━━━━━━━━━━━ 34s 42ms/step - accuracy: 0.8852 - loss: 0.2755 - val_accuracy: 0.8897 - val_loss: 0.1143
Epoch 6/20
806/806 ━━━━━━━━━━━━━━━━━━━━ 34s 42ms/step - accuracy: 0.8820 - loss: 0.3004 - val_accuracy: 0.8799 - val_loss: 0.1504
Epoch 7/20
806/806 ━━━━━━━━━━━━━━━━━━━━ 34s 42ms/step - accuracy: 0.8808 - loss: 0.3012 - val_accuracy: 0.7386 - val_loss: 0.3209
Epoch 8/20
806/806 ━━━━━━━━━━━━━━━━━━━━ 34s 42ms/step - accuracy: 0.7704 - loss: 0.3949 - val_accurac

...

323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
  PGD ε=0.07 → F1: 0.8320, Precision: 0.7526, Recall: 0.9302, MCC: 0.45828674141558534, Specificity: 0.4586021505376221, Balanced Acc: 0.6943838139966082

MI-FGSM Attack Evaluation:
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
 MI-FGSM ε=0.01 → F1: 0.9217, Precision: 0.8570, Recall: 0.9970, MCC: 0.7723828180180268, Specificity: 0.7053763440860026, Balanced Acc: 0.8511700302485576
 68/323 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

2025-08-27 13:04:06.810296: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
 MI-FGSM ε=0.03 → F1: 0.9204, Precision: 0.8565, Recall: 0.9947, MCC: 0.7680547502574361, Specificity: 0.7048387096774004, Balanced Acc: 0.8497626066984239
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
 MI-FGSM ε=0.05 → F1: 0.9123, Precision: 0.8483, Recall: 0.9868, MCC: 0.7415937559177095, Specificity: 0.6876344086021321, Balanced Acc: 0.8372132874952364
 68/323 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

2025-08-27 13:04:14.772944: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
 MI-FGSM ε=0.07 → F1: 0.8363, Precision: 0.7587, Recall: 0.9315, MCC: 0.47556905990537657, Specificity: 0.47553763440858937, Balanced Acc: 0.7035347197395915


In [16]:
# generate adversarial values for gan training

# experimental PGD code (GPU-optimized, batched)
def pgd_attack_tf_fast(model, x, y, epsilon=0.05, alpha=0.01, num_iter=10, clip_min=-1.5, clip_max=1.5):
    x = tf.convert_to_tensor(x, dtype=tf.float32)
    y = tf.reshape(tf.cast(y, tf.float32), (-1, 1))
    x_orig = tf.identity(x)
    x_adv = tf.identity(x)

    for _ in range(num_iter):
        with tf.GradientTape() as tape:
            tape.watch(x_adv)
            logits = model(x_adv, training=False)
            loss = tf.keras.losses.binary_crossentropy(y, logits)
            loss = tf.reduce_mean(loss)

        grad = tape.gradient(loss, x_adv)
        signed_grad = tf.sign(grad)
        x_adv = x_adv + alpha * signed_grad
        x_adv = tf.clip_by_value(x_adv, x_orig - epsilon, x_orig + epsilon)
        x_adv = tf.clip_by_value(x_adv, clip_min, clip_max)

    return x_adv

def batch_pgd_attack(model, X, y, attack_fn, batch_size=1024, **kwargs):
    adv_list = []
    X = tf.convert_to_tensor(X, dtype=tf.float32)
    y = tf.convert_to_tensor(y, dtype=tf.float32)
    for i in range(0, len(X), batch_size):
        X_batch = X[i:i+batch_size]
        y_batch = y[i:i+batch_size]
        x_adv_batch = attack_fn(model, X_batch, y_batch, **kwargs)
        adv_list.append(x_adv_batch)
    return tf.concat(adv_list, axis=0)

# Generate adversarial examples in batches to leverage GPU
X_adv_train = batch_pgd_attack(model, X_train, y_train, pgd_attack_tf_fast, epsilon=0.05, batch_size=1024)
print(f"X_adv_train shape: {X_adv_train.shape}")

X_adv_val = batch_pgd_attack(model, X_val, y_val, pgd_attack_tf_fast, epsilon=0.05, batch_size=1024)
print(f"X_adv_val shape: {X_adv_val.shape}")


X_adv_train shape: (206138, 95, 1)
X_adv_val shape: (41228, 95, 1)


In [17]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.callbacks import EarlyStopping

# ------------------------------
# Generator CNN (1D output)
# ------------------------------
class GeneratorCNN(Model):
    def __init__(self):
        super().__init__()
        self.conv1 = layers.Conv1D(64, 3, padding='same', activation='relu')
        self.bn1 = layers.BatchNormalization()
        self.conv2 = layers.Conv1D(128, 3, padding='same', activation='relu')
        self.bn2 = layers.BatchNormalization()
        self.conv3 = layers.Conv1D(64, 3, padding='same', activation='relu')
        self.bn3 = layers.BatchNormalization()
        # final output: 1 channel (same as clean input)
        self.output_layer = layers.Conv1D(1, 3, padding='same', activation='linear')

    def call(self, x, training=False):
        x = self.conv1(x)
        x = self.bn1(x, training=training)
        x = self.conv2(x)
        x = self.bn2(x, training=training)
        x = self.conv3(x)
        x = self.bn3(x, training=training)
        return self.output_layer(x)


# ------------------------------
# Discriminator CNN (1D input)
# ------------------------------
class DiscriminatorCNN(Model):
    def __init__(self):
        super().__init__()
        self.conv1 = layers.Conv1D(64, 3, padding='same', activation='relu')
        self.global_pool = layers.GlobalAveragePooling1D()
        self.dense1 = layers.Dense(64, activation='relu')
        self.output_layer = layers.Dense(1, activation='sigmoid')

    def call(self, x, training=False):
        x = self.conv1(x)
        x = self.global_pool(x)
        x = self.dense1(x)
        return self.output_layer(x)


# ------------------------------
# GAN Purifier CNN
# ------------------------------
class GANPurifierCNN(Model):
    def __init__(self, generator, discriminator, lambda_adv=0.5):
        super().__init__()
        self.generator = generator
        self.discriminator = discriminator
        self.lambda_adv = lambda_adv
        self.bce = tf.keras.losses.BinaryCrossentropy()
        self.l1 = tf.keras.losses.MeanAbsoluteError()
        self.d_optimizer = tf.keras.optimizers.Adam(1e-4)
        self.g_optimizer = tf.keras.optimizers.Adam(1e-4)

    def compile(self, **kwargs):
        super().compile(**kwargs)

    def train_step(self, data):
        x_clean, x_adv = data
        with tf.GradientTape(persistent=True) as tape:
            x_fake = self.generator(x_adv, training=True)
            real_pred = self.discriminator(x_clean, training=True)
            fake_pred = self.discriminator(x_fake, training=True)

            # Discriminator loss
            d_loss = self.bce(tf.ones_like(real_pred), real_pred) + \
                     self.bce(tf.zeros_like(fake_pred), fake_pred)

            # Generator loss (reconstruction + adversarial)
            g_loss = self.l1(x_clean, x_fake) + \
                     self.lambda_adv * self.bce(tf.ones_like(fake_pred), fake_pred)

            # Cosine similarity
            x_clean_flat = tf.reshape(x_clean, (tf.shape(x_clean)[0], -1))
            x_fake_flat = tf.reshape(x_fake, (tf.shape(x_fake)[0], -1))
            cos_sim = -tf.reduce_mean(
                tf.keras.losses.cosine_similarity(x_clean_flat, x_fake_flat)
            )

        # Compute gradients
        d_grads = tape.gradient(d_loss, self.discriminator.trainable_variables)
        g_grads = tape.gradient(g_loss, self.generator.trainable_variables)

        # Apply gradients
        self.d_optimizer.apply_gradients(zip(d_grads, self.discriminator.trainable_variables))
        self.g_optimizer.apply_gradients(zip(g_grads, self.generator.trainable_variables))

        return {"d_loss": d_loss, "g_loss": g_loss,
                "reconstruction_loss": self.l1(x_clean, x_fake),
                "cosine_similarity": cos_sim}





In [18]:
# ------------------------------
# Instantiate models
# ------------------------------
generator = GeneratorCNN()
discriminator = DiscriminatorCNN()

# Build models with dummy input (1 channel)
dummy_input = tf.zeros((1, 95, 1))
generator(dummy_input)
discriminator(dummy_input)

gan_cnn = GANPurifierCNN(generator, discriminator, lambda_adv=0.5)
gan_cnn.compile()

# ------------------------------
# Prepare dataset
# ------------------------------
batch_size = 512
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, X_adv_train))
train_dataset = train_dataset.shuffle(10000).batch(batch_size).prefetch(tf.data.AUTOTUNE)

early_stop = EarlyStopping(
    monitor="cosine_similarity",
    mode="max",              # higher cosine similarity is better
    patience=10,
    min_delta=1e-4,
    restore_best_weights=True,
    verbose=1
)

# ------------------------------
# Train
# ------------------------------
gan_cnn.fit(train_dataset, epochs=100, callbacks=[early_stop])


Epoch 1/100
403/403 ━━━━━━━━━━━━━━━━━━━━ 17s 26ms/step - cosine_similarity: 0.8812 - d_loss: 1.3756 - g_loss: 0.4489 - reconstruction_loss: 0.0991
Epoch 2/100
403/403 ━━━━━━━━━━━━━━━━━━━━ 11s 27ms/step - cosine_similarity: 0.9840 - d_loss: 1.3821 - g_loss: 0.3787 - reconstruction_loss: 0.0307
Epoch 3/100
403/403 ━━━━━━━━━━━━━━━━━━━━ 11s 27ms/step - cosine_similarity: 0.9895 - d_loss: 1.3819 - g_loss: 0.3728 - reconstruction_loss: 0.0248
Epoch 4/100
403/403 ━━━━━━━━━━━━━━━━━━━━ 11s 27ms/step - cosine_similarity: 0.9917 - d_loss: 1.3814 - g_loss: 0.3701 - reconstruction_loss: 0.0221
Epoch 5/100
403/403 ━━━━━━━━━━━━━━━━━━━━ 11s 27ms/step - cosine_similarity: 0.9928 - d_loss: 1.3805 - g_loss: 0.3689 - reconstruction_loss: 0.0207
Epoch 6/100
403/403 ━━━━━━━━━━━━━━━━━━━━ 11s 26ms/step - cosine_similarity: 0.9936 - d_loss: 1.3778 - g_loss: 0.3685 - reconstruction_loss: 0.0197
Epoch 7/100
403/403 ━━━━━━━━━━━━━━━━━━━━ 11s 26ms/step - cosine_similarity: 0.9941 - d_loss: 1.3706 - g_loss: 0.3698 -

In [19]:
# Generate purified outputs
x_purified_val = generator(X_adv_val, training=False)

# Cast to float32 for evaluation
x_purified_val = tf.cast(x_purified_val, tf.float32)
X_val_cast = tf.cast(X_val, tf.float32)

# L1 (Mean Absolute Error)
l1_loss = tf.reduce_mean(tf.abs(X_val_cast - x_purified_val))

# L2 (Mean Squared Error)
l2_loss = tf.reduce_mean(tf.square(X_val_cast - x_purified_val))

# Normalize along sequence dimension
X_val_norm = tf.nn.l2_normalize(tf.squeeze(X_val_cast, axis=-1), axis=1)           
x_purified_norm = tf.nn.l2_normalize(tf.squeeze(x_purified_val, axis=-1), axis=1)  

# Cosine similarity
cos_sim = tf.reduce_mean(tf.reduce_sum(X_val_norm * x_purified_norm, axis=1))

print(f"🧪 Purifier Evaluation Metrics:")
print(f"✅ Cosine Similarity: {cos_sim:.6f}")
print(f"   ➤ L1 Loss (MAE): {l1_loss:.6f}")
print(f"   ➤ L2 Loss (MSE): {l2_loss:.6f}")


W0000 00:00:1756300123.499227      36 gpu_utils.cc:68] Failed to allocate memory for convolution redzone checking; skipping this check. This is benign and only means that we won't check cudnn for out-of-bounds reads and writes. This message will only be printed once.


🧪 Purifier Evaluation Metrics:
✅ Cosine Similarity: 0.994910
   ➤ L1 Loss (MAE): 0.016329
   ➤ L2 Loss (MSE): 0.001034


## GAN+Adv Training evaluation

In [20]:
def evaluate_model(model, generator, adv_model, X, y, epsilon_list=[0.01, 0.03, 0.05, 0.07]):
    import tensorflow as tf
    import numpy as np

    X_clean = X
    x_clean_tf = tf.convert_to_tensor(X_clean, dtype=tf.float32)
    y_tf = tf.convert_to_tensor(y, dtype=tf.float32)

    # ----------- Clean Data Evaluation (no adversarial) -----------
    print("✅ Clean Evaluation (GAN purified):")
    x_purified_clean = generator(x_clean_tf, training=False)
    y_pred_clean_prob = adv_model.predict(x_purified_clean)
    y_pred_clean = (y_pred_clean_prob > 0.5).astype(int).flatten()
    clean_metrics = evaluate_metrics(y, y_pred_clean)
    print(f"Clean → F1: {clean_metrics['f1']:.4f}, Precision: {clean_metrics['precision']:.4f}, Recall: {clean_metrics['recall']:.4f}, MCC: {clean_metrics.get('mcc', 'N/A')}, Specificity: {clean_metrics.get('specificity', 'N/A')}, Balanced Acc: {clean_metrics.get('balanced_accuracy', 'N/A')}")

    # ----------- FGSM Attack Evaluation -----------
    print("\nFGSM Attack Evaluation with GAN Purification")
    fgsm_results = []
    for eps in epsilon_list:
        x_adv = fgsm_attack_tf(model, x_clean_tf, y_tf, epsilon=eps )
        x_adv_tf = tf.convert_to_tensor(x_adv.numpy(), dtype=tf.float32)
        x_purified = generator(x_adv_tf, training=False)
        y_pred_prob = adv_model.predict(x_purified)
        y_pred = (y_pred_prob > 0.5).astype(int).flatten()

        adv_metrics = evaluate_metrics(y, y_pred)
        fgsm_results.append({'epsilon': eps, 'adversarial': adv_metrics})

        print(f"  FGSM ε={eps:.2f} → F1: {adv_metrics['f1']:.4f}, Precision: {adv_metrics['precision']:.4f}, Recall: {adv_metrics['recall']:.4f}, MCC: {adv_metrics.get('mcc', 'N/A')}, Specificity: {adv_metrics.get('specificity', 'N/A')}, Balanced Acc: {adv_metrics.get('balanced_accuracy', 'N/A')}")

    # ----------- PGD Attack Evaluation -----------
    print("\nPGD Attack Evaluation with GAN Purification:")
    pgd_results = []
    for eps in epsilon_list:
        alpha = eps * 0.2
        x_adv = pgd_attack_tf(model, x_clean_tf, y_tf, epsilon=eps, alpha=alpha, num_iter=10)
        x_adv_tf = tf.convert_to_tensor(x_adv.numpy(), dtype=tf.float32)
        x_purified = generator(x_adv_tf, training=False)
        y_pred_prob = adv_model.predict(x_purified )
        y_pred = (y_pred_prob > 0.5).astype(int).flatten()

        adv_metrics = evaluate_metrics(y, y_pred)
        pgd_results.append({'epsilon': eps, 'adversarial': adv_metrics})

        print(f"  PGD ε={eps:.2f} → F1: {adv_metrics['f1']:.4f}, Precision: {adv_metrics['precision']:.4f}, Recall: {adv_metrics['recall']:.4f}, MCC: {adv_metrics.get('mcc', 'N/A')}, Specificity: {adv_metrics.get('specificity', 'N/A')}, Balanced Acc: {adv_metrics.get('balanced_accuracy', 'N/A')}")

      # ----------- MI-FGSM Attack Evaluation -----------
    print("\nMI-FGSM Attack Evaluation with GAN Purification:")
    mifgsm_results = []
    for eps in epsilon_list:
        alpha = eps * 0.2
        steps = 10
        decay = 1.0

        x_adv = mi_fgsm_attack(model, x_clean_tf, y_tf, epsilon=eps, alpha=alpha, steps=steps, decay=decay)
        x_adv_tf = tf.convert_to_tensor(x_adv.numpy(), dtype=tf.float32)
        x_purified = generator(x_adv_tf, training=False)
        y_pred_prob = adv_model.predict(x_purified )
        y_pred = (y_pred_prob > 0.5).astype(int).flatten()

        adv_metrics = evaluate_metrics(y, y_pred)
        mifgsm_results.append({'epsilon': eps, 'adversarial': adv_metrics})

        print(f"  MI-FGSM ε={eps:.2f} → F1: {adv_metrics['f1']:.4f}, Precision: {adv_metrics['precision']:.4f}, Recall: {adv_metrics['recall']:.4f}, MCC: {adv_metrics.get('mcc', 'N/A')}, Specificity: {adv_metrics.get('specificity', 'N/A')}, Balanced Acc: {adv_metrics.get('balanced_accuracy', 'N/A')}")

    return fgsm_results, pgd_results, mifgsm_results


In [21]:
fgsm_full, pgd_full, mi_full = evaluate_model(model= model,generator=generator,adv_model= model, X = X_test, y = y_test)


✅ Clean Evaluation (GAN purified):
 67/323 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

2025-08-27 13:08:44.246461: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Clean → F1: 0.9204, Precision: 0.8542, Recall: 0.9977, MCC: 0.7685060772148689, Specificity: 0.6983870967741748, Balanced Acc: 0.8480549420412546

FGSM Attack Evaluation with GAN Purification
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
  FGSM ε=0.01 → F1: 0.9200, Precision: 0.8540, Recall: 0.9971, MCC: 0.7672482569044973, Specificity: 0.6981182795698737, Balanced Acc: 0.8476169050802154
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
  FGSM ε=0.03 → F1: 0.9195, Precision: 0.8535, Recall: 0.9967, MCC: 0.7656555797237455, Specificity: 0.6970430107526694, Balanced Acc: 0.8468515494024467
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
  FGSM ε=0.05 → F1: 0.9167, Precision: 0.8505, Recall: 0.9941, MCC: 0.7563556514554237, Specificity: 0.6905913978494438, Balanced Acc: 0.8423353224255569
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
  FGSM ε=0.07 → F1: 0.9116, Precision: 0.8478, Recall: 0.9857, MCC: 0.7389686732915264, Specificity: 0.6865591397849278, Balanced Acc: 0.8361

2025-08-27 13:08:51.783087: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
  PGD ε=0.01 → F1: 0.9200, Precision: 0.8540, Recall: 0.9971, MCC: 0.7672482569044973, Specificity: 0.6981182795698737, Balanced Acc: 0.8476169050802154
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
  PGD ε=0.03 → F1: 0.9194, Precision: 0.8534, Recall: 0.9965, MCC: 0.7651913194854297, Specificity: 0.6967741935483683, Balanced Acc: 0.846641233710574
 69/323 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

...

323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
  PGD ε=0.05 → F1: 0.9159, Precision: 0.8496, Recall: 0.9933, MCC: 0.753629550335994, Specificity: 0.6887096774193363, Balanced Acc: 0.8410149267618923
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
  PGD ε=0.07 → F1: 0.9088, Precision: 0.8430, Recall: 0.9857, MCC: 0.7301857556599881, Specificity: 0.6749999999999818, Balanced Acc: 0.830364733566106

MI-FGSM Attack Evaluation with GAN Purification:
 67/323 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

2025-08-27 13:09:02.667864: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
  MI-FGSM ε=0.01 → F1: 0.9200, Precision: 0.8540, Recall: 0.9971, MCC: 0.7672482569044973, Specificity: 0.6981182795698737, Balanced Acc: 0.8476169050802154
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
  MI-FGSM ε=0.03 → F1: 0.9195, Precision: 0.8537, Recall: 0.9964, MCC: 0.765527814692588, Specificity: 0.6975806451612716, Balanced Acc: 0.8469685524273034
 67/323 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

2025-08-27 13:09:08.122753: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
  MI-FGSM ε=0.05 → F1: 0.9165, Precision: 0.8500, Recall: 0.9944, MCC: 0.7558782604362322, Specificity: 0.6892473118279384, Balanced Acc: 0.8418150935942486
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
  MI-FGSM ε=0.07 → F1: 0.9103, Precision: 0.8461, Recall: 0.9850, MCC: 0.734816532854913, Specificity: 0.6827956989247128, Balanced Acc: 0.8338830475798606
